# 02-02 : Tutorial Test

User remote failed with `{'reason': "Runtime error: 'K8sHelper' object has no attribute 'v1api'"}`, follow the references below as an experiment.

## References

- [GitHub Discussion](https://github.com/mlrun/mlrun/discussions/5579)
- [Model monitoring tutorial](https://docs.mlrun.org/en/stable/tutorials/05-model-monitoring.html)


In [1]:
import os

In [2]:
# set the api url
mlrun_api_server = 'dragon'
mlrun_api_port = 8080
os.environ["MLRUN_DBPATH"] = f"http://{mlrun_api_server}:{mlrun_api_port}"

In [3]:
import mlrun
import uuid

project_name = "tutorial"
project = mlrun.get_or_create_project(project_name, "./")

> 2025-05-12 13:07:02,575 [warning] Client version with higher version than server version isn't supported, align your client to the server version: {"parsed_client_version":"Version(major=1, minor=7, patch=2, prerelease=None, build=None)","parsed_server_version":"Version(major=1, minor=6, patch=2, prerelease=None, build=None)"}
> 2025-05-12 13:07:02,595 [info] Project loaded successfully: {"project_name":"tutorial"}


In [4]:
#project.set_model_monitoring_credentials(None, "v3io", "v3io", "v3io")
#project.enable_model_monitoring(base_period=2)

In [5]:
# Download the training set
import pandas as pd

train_set = pd.read_csv(
    "https://s3.us-east-1.wasabisys.com/iguazio/data/iris/iris_dataset.csv"
)

In [6]:
model_name = "RandomForestClassifier"
project.log_model(
    model_name,
    model_file="src/model.pkl",
    training_set=train_set,
    framework="sklearn",
    label_column="label",
)

In [7]:
# Import the serving function
serving_fn = mlrun.import_function(
    "hub://v2_model_server", project=project_name, new_name="serving"
)

serving_fn.add_model(
    model_name, model_path=f"store://models/tutorial/RandomForestClassifier:latest"
)

# enable monitoring on this serving function
#serving_fn.set_tracking()

serving_fn.spec.build.requirements = ["scikit-learn~=1.5.1"]

# Deploy the serving function
project.deploy_function(serving_fn)

> 2025-05-12 11:07:27,522 [info] Starting remote function deploy
> 2025-05-12 11:07:27,569 [error] error submitting nuclio deploy task: details: {'reason': "Runtime error: 'K8sHelper' object has no attribute 'v1api'"}, caused by: 400 Client Error: Bad Request for url: http://localhost:8080/api/v1/projects/tutorial/nuclio/serving/deploy


OSError: error: cannot submit deploy, details: {'reason': "Runtime error: 'K8sHelper' object has no attribute 'v1api'"}, caused by: 400 Client Error: Bad Request for url: http://localhost:8080/api/v1/projects/tutorial/nuclio/serving/deploy